In [1]:
import ee
import geojson

In [2]:
#ee.Authenticate() # Do this just once
ee.Initialize()

In [3]:
## Select collection
loc = ee.Geometry.Point(-1.9, 55)
collection = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(loc)
    .filterDate('2021-02-13', '2021-02-14')
)
bands = collection.select(['B3', 'B11'])

In [4]:
## Show available data
print(bands.aggregate_array('system:index').getInfo())

['20210213T112151_20210213T112445_T30UWF', '20210213T112151_20210213T112445_T30UWG']


In [9]:
## Polygon - boundary
with open("C:/Users/frans/Documents/NCL/R2D2/data/north_east_4326.geojson") as json_file:
   gjson = geojson.load(json_file)

## get gjson features
features = gjson['features']

## make Earth Engine FeatureCollection of features
fc = ee.FeatureCollection(features)

In [11]:
## clip image func
def clip_img(var):
    clip = var.clip(fc)  # fc is a feature collection
    return clip

In [12]:
## clip every image in collection
clip = bands.map(clip_img)

nimg = clip.toList(clip.size().getInfo()).size().getInfo()
area = fc.geometry()  

## export result                                 
for i in range(nimg):
    img = ee.Image(clip.toList(nimg).get(i))
    date = img.date().format('yyyy_MM_dd_HH').getInfo()

    task = ee.batch.Export.image.toDrive(img.toFloat(), 
                                          description=date,
                                          folder='gee',
                                          fileNamePrefix= date,
                                          region = area,
                                          # fileFormat = 'TFRecord',
                                          maxPixels = 1.0E13)
    task.start()